In [4]:
using Fermionic

# Operators

We then create elements of the type Op, with dimension n. 
This structure contains the operators as methods, with its corresponding names. These are

$c_i \rightarrow cm(i)\\
c_i^{\dagger} \rightarrow cmd(i)\\
c_ic_j \rightarrow cmcm(i,j)\\
c_i^{\dagger}c_j^\dagger \rightarrow cdcd(i,j)\\
c_ic_j^\dagger \rightarrow cmcd(i,j)\\
c_i^\dagger c_j \rightarrow cdcm(i)$

One of the great advantages of defining these operators as types, is that it is possible to simultaneously define operators for different dimensions. This can be useful for a number of application, for example for comparing results in different dimensions without re-running the core program each time you switch dimension.

In Object oriented programming, one would define a class Operator. Here, we are defining a type (like Int or Float) and method associated to that kind of type. The big advantage of doing this over defining classes, is that Julia uses **multiple dispatch**. What that means, is that a method can be defined for different input types, and Julia will run the correct method depending on all the input's types. Whereas in object oriented programming, programs select the method from the type of the first input only. This will be very useful for defining states, where we can work both with normal arrays and sparse arrays.

You can do, for instance

op4 = Op(4)

op6 = Op(6)

and op4 will be an element of the type ::Op, and will have all its methods defined for fermionic operators in dimension 4.

In [8]:
op4 = Op(4);

All the operators can be represented by matrices. Lets first look at the basis

In [10]:
basis(op4)

16×4 SparseMatrixCSC{Float64,Int64} with 32 stored entries:
  [9 , 1]  =  1.0
  [10, 1]  =  1.0
  [11, 1]  =  1.0
  [12, 1]  =  1.0
  [13, 1]  =  1.0
  [14, 1]  =  1.0
  [15, 1]  =  1.0
  [16, 1]  =  1.0
  [5 , 2]  =  1.0
  [6 , 2]  =  1.0
  [7 , 2]  =  1.0
  [8 , 2]  =  1.0
  ⋮
  [8 , 3]  =  1.0
  [11, 3]  =  1.0
  [12, 3]  =  1.0
  [15, 3]  =  1.0
  [16, 3]  =  1.0
  [2 , 4]  =  1.0
  [4 , 4]  =  1.0
  [6 , 4]  =  1.0
  [8 , 4]  =  1.0
  [10, 4]  =  1.0
  [12, 4]  =  1.0
  [14, 4]  =  1.0
  [16, 4]  =  1.0

A sparse matrix is a way of representing a matrix in which one inidicates (row, col) = value. This is really handy for defining matrices with many zeros, as no unnecessary memory is used for storing empty elements. The drawback of the sparse representation is that it's hard to understand what is really doing. We can use the function Matrix() in order to convert a sparse matrix into a normal matrix. I recommend doing this only in order to see what is happening, not for doing operations. This is because both memory and time increase enourmusly when operations are done with full matrices instead of sparse.

In [11]:
Matrix(basis(op4))

16×4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  1.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  1.0
 0.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0
 1.0  0.0  1.0  0.0
 1.0  0.0  1.0  1.0
 1.0  1.0  0.0  0.0
 1.0  1.0  0.0  1.0
 1.0  1.0  1.0  0.0
 1.0  1.0  1.0  1.0

Ok, so now we can see that this basis is composed of 16 4-dimensional arrays, each representing a posible fermionic slater determinant in 4 dimensions. Each mode can be occupied by 1 or by 0 fermions (due to its antisimmetric nature). A general fermionic state will be a linear combination of these slater determinants.

The first state is the vacuum. The second corresponds to 